In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import os.path
import time
import random
import pickle

In [20]:
# Create negative image library
negative_images = []

for dirpath, dirnames, filenames in os.walk("./faces94"):
    for filename in [f for f in filenames if f.endswith(".jpg")]:
        image_data = plt.imread(os.path.join(dirpath, filename))
        negative_images.append([len(negative_images), 0, image_data])
        
print(len(negative_images), negative_images[1][0], negative_images[1][1], negative_images[0][2].shape)

# Create positive image library
positive_images = []

for filename in os.listdir("./charles"):
    if filename.endswith(".jpg"):
        image_data = plt.imread(os.path.join("./charles", filename))
        positive_images.append([len(negative_images) + len(positive_images), 1, image_data])

print(len(positive_images), positive_images[0][0], positive_images[0][1], positive_images[0][2].shape)

3059 1 0 (200, 180, 3)
71 3059 1 (200, 180, 3)


C:\Users\Charles\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:764: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [22]:
# Generate train, validation and test sets
FRAC_TEST = 0.1
FRAC_TRAIN = 0.8

def gen_cross_validation(positive_train_set, negative_train_set, frac_train=0.8):
    """Generate a cross-validation fold."""
    
    n_pos = len(positive_train_set)
    n_neg = len(negative_train_set)
    random.shuffle(positive_train_set)
    random.shuffle(negative_train_set)
    
    train = positive_train_set[:int(frac_train*n_pos)] + negative_train_set[:int(frac_train*n_neg)]
    valid = positive_train_set[int(frac_train*n_pos):] + negative_train_set[int(frac_train*n_neg):]
    random.shuffle(train)
    random.shuffle(valid)
    
    return train, valid
    
# Generate train and test split
n_neg = len(negative_images)
n_pos = len(positive_images)
random.shuffle(positive_images)
random.shuffle(negative_images)

negative_train_set = negative_images[int(FRAC_TEST*n_neg):]
positive_train_set = positive_images[int(FRAC_TEST*n_pos):]
test_set = negative_images[:int(FRAC_TEST*n_neg)] + positive_images[:int(FRAC_TEST*n_pos)]

print(len(negative_train_set), len(positive_train_set), len(test_set), n_neg + n_pos)

2754 64 312 3130


In [23]:
def save_struct(filename, struct):
    file = open(filename, 'wb') 
    pickle.dump(struct, file)
    file.close()

def load_struct(filename):
    file = open(filename, 'rb')  
    struct = pickle.load(file)  
    file.close()
    return struct

# Save the permanent datasets
save_struct('negative_train_set', negative_train_set)
save_struct('positive_train_set', positive_train_set)
save_struct('test_set', test_set)

In [ ]:
dataset = np.load('dataset.npy')

for i in range(len(dataset)):
    plt.imshow(dataset[i])
    plt.show()